In [1]:
from collections import deque
from itertools import cycle
import random
from lgsvl.geometry import Vector
import lgsvl.utils

# All coordinates in the API return values in the Unity coordinate system. 
# This coordinate system uses meters as a unit of distance and is a left-handed coordinate system - 
# x points left, z points forward, and y points up.

###
#
# These are the fields used in BSM message
#
###
BSM_PART_1_FIELDS = [
    'msgCnt', # Done
    'id', # Done
    'secMark', # Done
    'latPos',  # Done
    'longPos', # Done
    'elevation',    # Done
    'semiMajor', # Not implemented
    'semiMinor', # Not implemented
    'orientation', # Not implemented
    'transmission', # Not implemented
    'heading', # Done
    'speed', # Done
    'steering', # Not implemented
    'longAccel', # TODO
    'latAccel', # TODO
    'vertAccel', # TODO
    'yawRate', # TODO
    'wheelBrakes', # ?
    'traction', # ?
    'abs', # Not implemented
    'scs', # Not implemented
    'brakeBoost', # Some threshold probably needed, or keep track of 'braking' history. Also, same problem as angle?
    'auxBrake', # AuxBrake, question on access due to VehicleControl
    'width', # TODO, look at bounding box
    'length' # TODO, look at bounding box
]

class BasicSafetyMessage:
    __slots__ = BSM_PART_1_FIELDS
    def __init__(self, **kwargs):
        # Iterate through each field and set to None
        for field in self.__slots__:
            setattr(self, field, kwargs.setdefault(field, None))

    @classmethod
    def to_json(bsm):
        return json.dumps(bsm.__dict__)

def BSM_part_1_gen(agent):
    states = deque(maxlen=2)
    times = deque(maxlen=2)
    message_count_gen = cycle(range(127))
    msg_id = random.randint(0, 2**32-1)
    states.appendleft(agent.state)
    states.appendleft(agent.state)
    times.appendleft(agent.simulator.current_time)
    times.appendleft(agent.simulator.current_time)

    agent_forward = lgsvl.utils.transform_to_forward(states[0].transform)
    agent_forward = agent_forward * (1 / agent_forward.magnitude()) # Get unit vector

    agent_up = lgsvl.utils.transform_to_up(states[0].transform)
    agent_up = agent_up * (1 / agent_up.magnitude())

    agent_left = -1 * lgsvl.utils.transform_to_right(states[0].transform) # Make if left to match unity coord system
    agent_left = agent_left * (1 / agent_left.magnitude())

    # Translational/Rotational acceleration
    try:
        translational_acc = (states[0].velocity - states[1].velocity) * (1 / (times[0] - times[1]))
        rotational_acc = (states[0].angular_velocity - states[1].angular_velocity) * (1 / (times[0] - times[1]))
    except ZeroDivisionError:
        translational_acc = Vector(x=0, y=0, z=0)
        rotational_acc = Vector(x=0, y=0, z=0)
        
    while True:
        bsm = BasicSafetyMessage()
        gps_data = agent.simulator.map_to_gps(states[0].transform)

        # A sequence number within a stream of messages with the same DSRC msgID and from the same sender.
        bsm.msgCnt = next(message_count_gen)
        
        # 4 octet random device identifier. Changes periodically to ensure the overall anonymity of the vehicle
        bsm.id = msg_id

        # Represents the milliseconds within a minute –units of milliseconds
        bsm.secMark = int((agent.simulator.current_time % 1) * 1000)

        # The geographic latitude of an object, expressed in 1/10th integer microdegrees
        bsm.latPos = int(gps_data.latitude * 10_000_000)
        # The geographic longitude of an object, expressed in 1/10th integer microdegrees
        bsm.longPos = int(gps_data.longitude * 10_000_000)
        # Geographic position above or below the reference ellipsoid (typically WGS-84). The number has a resolution of 1 decimeter
        bsm.elevation = int(gps_data.altitude * 10)

        # Not implemented
        bsm.semiMajor = 0
        bsm.semiMinor = 0
        bsm.orientation = 0 # This is the orientation of the ellipsoid, not the vehicle

        # current state of the vehicle transmission
        bsm.transmission = 7 # Enumerated value, transmission is 'unavailable', so 7 is used

        # vehicle speed expressed in unsigned units of 0.02 meters per second
        bsm.speed = (agent.state.speed // 0.02)
        #current heading of the sending device, expressed in unsigned units of 0.0125 degrees from North
        bsm.heading = (gps_data.orientation % 360) // 0.0125
        # The angle of the driver’s steering wheel, with LSB units of 1.5 degrees, +127 to be used for unavailable
        bsm.steering = 127

        print(translational_acc)
        print(agent_forward)
        print(bsm.speed)
        # Signed acceleration of the vehicle along some known axis in units of 0.01 meters per second squared along the Vehicle Longitudinal axis
        bsm.longAccel = (agent_forward * translational_acc).magnitude() // 0.01
        # Signed acceleration of the vehicle along some known axis in units of 0.01 meters per second squared along the Vehicle Lateral axis
        bsm.latAccel = (agent_left * translational_acc).magnitude() // 0.01
        # Signed vertical acceleration of the vehicle along the vertical axis in units of 0.02 G
        bsm.vertAccel = (agent_up * translational_acc).magnitude() // 0.02

        # Yaw Rate of the vehicle, a signed value (to the right being positive) expressed in 0.01 degrees per second
        bsm.yawRate = -1 * states[0].angular_velocity.x

        # 'wheelBrakes', # ?
        # 'traction', # ?
        # 'abs', # Not implemented
        # 'scs', # Not implemented
        # 'brakeBoost', # Some threshold probably needed, or keep track of 'braking' history. Also, same problem as angle?
        # 'auxBrake', # AuxBrake, question on access due to VehicleControl
        
#         'width', # TODO, look at bounding box
#         'length' # TODO, look at bounding box

        yield bsm


        states.appendleft(agent.state)
        times.appendleft(agent.simulator.current_time)
        agent_forward = lgsvl.utils.transform_to_forward(states[0].transform)
        agent_forward = agent_forward * (1 / agent_forward.magnitude()) # Get unit vector

        agent_up = lgsvl.utils.transform_to_up(states[0].transform)
        agent_up = agent_up * (1 / agent_up.magnitude())

        agent_left = -1 * lgsvl.utils.transform_to_right(states[0].transform) # Make if left to match unity coord system
        agent_left = agent_left * (1 / agent_left.magnitude())

        # Translational/Rotational acceleration
        try:
            translational_acc = (states[0].velocity - states[1].velocity) * (1 / (times[0] - times[1]))
            rotational_acc = (states[0].angular_velocity - states[0].angular_velocity) * (1 / (times[0] - times[1]))
        except ZeroDivisionError:
            translational_acc = Vector(x=0, y=0, z=0)
            rotational_acc = Vector(x=0, y=0, z=0)


In [2]:
import os
import lgsvl

INTERVAL = 0.1

sim = lgsvl.Simulator(os.environ.get("SIMULATOR_HOST", "127.0.0.1"), 8181)

print("Version =", sim.version)
print("Current Time =", sim.current_time)
print("Current Frame =", sim.current_frame)

print("Current Scene = {}".format(sim.current_scene))

# Loads the named map in the connected simulator. The available maps can be set up in web interface
if sim.current_scene == "BorregasAve":
  sim.reset()
else:
  sim.load("BorregasAve")

print("Current Scene = {}".format(sim.current_scene))

# This will print out the position and rotation vectors for each of the spawn points in the loaded map
spawns = sim.get_spawn()
for spawn in sim.get_spawn():
  print(spawn)

# Get a state from the simulator and spawn a vehicle
spawns = sim.get_spawn()

state = lgsvl.AgentState()
state.transform = spawns[0]

sim.load('BorregasAve')

forward = lgsvl.utils.transform_to_forward(spawns[0])

# Agents can be spawned with a velocity. Default is to spawn with 0 velocity
state.velocity = 20 * forward
a = sim.add_agent("Lincoln2017MKZ (Apollo 5.0)", lgsvl.AgentType.EGO, state)

Version = 2020.06
Current Time = 21.9999995082617
Current Frame = 2200
Current Scene = BorregasAve
Current Scene = BorregasAve
Spawn(position=Vector(-50.3400077819824, -1.03600001335144, -9.03001022338867), rotation=Vector(0, 104.823379516602, 0), destinations=[])
Spawn(position=Vector(386.800048828125, -7.73000001907349, 106.700073242188), rotation=Vector(0, 191.806182861328, 0), destinations=[])


In [3]:
a.bsm_gen = BSM_part_1_gen(a)

In [7]:
sim.run(.1)

In [5]:
for step in range(100):
    sim.run(INTERVAL)
    bsm = next(a.bsm_gen)
#     print(f'{step:03d}: {bsm.longAccel}', bsm.speed)

Vector(0, 0, 0)
Vector(0.9662507126528467, -0.030037436421900914, -0.25584626772940466)
958.0
000: 0.0 958.0
Vector(-0.429210672435598, 1.1401629702782796, 0.08150577727339063)
Vector(0.9662810468536013, -0.02769591537662068, -0.25599584911275314)
956.0
001: 41.0 956.0
Vector(-0.2941131657529423, 0.4911828150864423, 0.08779049115907088)
Vector(0.9664511288193148, -0.018714321949400394, -0.25616789369053694)
954.0
002: 28.0 954.0
Vector(-0.32463074456105306, 1.2342879452025366, 0.09442329617793116)
Vector(0.9665208167936793, -0.01736189972327202, -0.25600014676261107)
953.0
003: 31.0 953.0
Vector(-0.317459113540779, 0.1860860031713385, 0.11367321268478806)
Vector(0.9665737211471975, -0.016422065964029706, -0.25586237968316616)
951.0
004: 30.0 951.0
Vector(-0.3989982694163052, 0.641667261160958, 0.11858463552417733)
Vector(0.9666561928647552, -0.011025401886550124, -0.2558402730405389)
949.0
005: 38.0 949.0
Vector(-0.5127716179073402, 1.068076142700177, 0.1323890715626275)
Vector(0.96670

Vector(-0.2752685608407463, -0.014488399352611613, 0.08290767854994018)
Vector(0.9662227320951365, -0.01722528022766911, -0.25713210943732445)
871.0
051: 26.0 871.0
Vector(-0.2865409915116731, -0.016441345582341838, 0.09924411995507318)
Vector(0.9662318830454537, -0.017339471169353932, -0.2570900443930178)
869.0
052: 27.0 869.0
Vector(-0.26302338234406847, -0.0684800759363234, 0.10183334578195619)
Vector(0.9662392979633985, -0.01771805520234771, -0.2570363585002042)
868.0
053: 25.0 868.0
Vector(-0.2790641847045327, -0.03734767520459373, 0.09696483828793419)
Vector(0.9662454653198667, -0.018344025131694502, -0.256969253979464)
867.0
054: 27.0 867.0
Vector(-0.2607154904464601, -0.07409572766935345, 0.08018493831577769)
Vector(0.9662528556469006, -0.018616016419086938, -0.2569219003644772)
865.0
055: 25.0 865.0
Vector(-0.28572083158137385, 0.14391720616631387, 0.07574558427363544)
Vector(0.9662689513475706, -0.017885907615943154, -0.2569132304308601)
864.0
056: 27.0 864.0
Vector(-0.270824

In [6]:
a.state

{'transform': 'Transform(position=Vector(137.506134033203, -4.24356412887573, -59.1194534301758), rotation=Vector(0.668341517448425, 104.90705871582, 358.659912109375))', 'velocity': 'Vector(15.3936538696289, -0.185055702924728, -4.12914991378784)', 'angular_velocity': 'Vector(-0.00162000232376158, 0.000419774005422369, 0.00767719279974699)'}

In [8]:
a.state

{'transform': 'Transform(position=Vector(139.043792724609, -4.26174259185791, -59.5318069458008), rotation=Vector(0.641975402832031, 104.908996582031, 358.633850097656))', 'velocity': 'Vector(15.3627405166626, -0.176293879747391, -4.12198829650879)', 'angular_velocity': 'Vector(-0.00398117443546653, 0.000367188738891855, 0.00451917387545109)'}